This is everything I need to perform temporal instanton anlaysis on a network:


| Var name        | Description                                  |
|:---------------:|:---------------------------------------------|
| `Ridx`            | Vector: indices of wind nodes                |
| `Y`               | admittance matrix                            |
| `G0`              | Conventional generation                      |
| `P0`              | Renewable forecast                           |
| `D0`              | Demand                                       |
| `Sb`              | System base MVA                              |
| `ref`             | index of angle reference bus                 |
| `lines`           | vector of tuples: (from,to)                  |
| `res`             | pu line resistances                          |
| `reac`            | pu line reactances                           |
| `k`               | generator participation factors              |
| `line_lengths`    | line lengths in meters                       |
| `line_conductors` | vector of strings containing conductor names |
| `Tamb`            | system ambient temperature                   |
| `T0`              | Line initial temperature                     |
| `int_length`      | length of each time interval in seconds      |

I need to obtain all ingredients in the table above for the 3000-node Polish network. Here is the particular Polish network scenario I am using: summer morning peak data for the 2008 Polish system.

In [2]:
using MatpowerCases
mpc = loadcase("case3120sp",describe=false)
print(mpc["docstring"])

 CASE3120SP  Power flow data for Polish system - summer 2008 morning peak.
    Please see CASEFORMAT for details on the case file format.
 
    This case represents the Polish 400, 220 and 110 kV networks during
    summer 2008 morning peak conditions.
 
    This data was graciously provided by, and is distributed with the
    permission of, Roman Korab <roman.korab@polsl.pl>.


## Ridx

I need a vector of wind nodes for `case3120sp`. I think the most sensible thing to do is convert small conventional generators into wind farms. That way I am not fundamentally disturbing the generator topology.

How many wind nodes should there be? Let's look at the RTS-96 network I've been using.

In [3]:
include("../src/TemporalInstanton.jl")
using TemporalInstanton

Ridx,Y,
Gp,Dp,Rp,
Sb,ref,lines,
res,reac,k,
line_lengths,line_conductors = load_rts96_data();

  Please email questions, comments, and significant results to: Robert.C.Green@gmail.com.  Thanks!


In [9]:
Gp # length = no. of nodes
Gp[find(Gp)] # length = no. of generators
sum(Gp)

89.979

In [26]:
Rp # length = no. of nodes
Rp[find(Rp)] # length = no. of wind farms
hcat(["mean:";"min:";"max:";"sum:"],
    round([mean(Rp);minimum(Rp[find(Rp)]);maximum(Rp);sum(Rp)],4))

4x2 Array{Any,2}:
 "mean:"  0.0585
 "min:"   0.0146
 "max:"   0.8194
 "sum:"   4.2711

The RTS-96 I use has

* 33 generators with a total of 90.0 pu generation.
* 18 wind farms with a total of 4.3 pu generation
    * Smallest: 1.46 MW
    * Largest: 81.9 MW

So the network has approximately 4.5% wind penetration.

What if I converted all Polish generators smaller than 82 MW into wind farms?

The Polish grid has 505 generators spread across 349 nodes. The first step in processing the generation data is to merge generators that share nodes.

In [154]:
Sb = mpc["baseMVA"] # "100.0"
bus_i = mpc["bus"][:,1]
genBus = mpc["gen"][:,1]
Gp_long = mpc["gen"][:,2] # in MW
Gp = zeros(length(bus_i))
for i in bus_i
    Gp[i] = sum(Gp_long[find(genBus.==i)])/Sb # in pu
end

Now consider nodes whose generation is less than some cutoff:

In [156]:
cutoff = 0.82 # 82 MW
# cutoff = 0.41 # 41 MW
ratio1 = round(sum(Gp[find(Gp)].<cutoff)/length(find(Gp)),4)
println("wind farms/conv. gen sites: $ratio1")

f(x) = x < cutoff
#g(x) = x >= cutoff
ratio2 = round(sum(filter(f,Gp[find(Gp)]))/sum(Gp),3)
println("wind penetration: $(ratio2*100)%")

wind farms/conv. gen sites: 0.7012
wind penetration: 10.2%


There are 241 nodes with generation, so an 82 MW cutoff would convert 71% of generators into wind farms.

In terms of total generation, the 82 MW cutoff would yield a wind fleet of 21.6 pu and leave 190.8 pu of conventional generation. This means wind penetration would be 10%. I think this is a reasonably interesting scenario, so I will move forward with an 82 MW cutoff.

This block of code converts generators smaller than 82 MW into wind nodes:

In [157]:
Rp = zeros(length(Gp))
for i in 1:length(Gp)
    if Gp[i] < 0.82
        Rp[i] = Gp[i]
        Gp[i] = 0
    end
end

## Line lengths

I can't find data on Polish grid line lengths anywhere online, and the guy who added Polish grid data to Matpower suggests length data [may not be public](https://www.mail-archive.com/matpower-l@cornell.edu/msg00785.html).

So for the purposes of showing that the temporal instanton analysis algorithm scales to 3000-node networks, I will sample RTS-96 line lengths to generate a vector of line lenghts for the polish grid. This vector will be saved (not randomly generated with each run).

In [166]:
using HDF5, JLD
# use RTS-96 line lengths to generate similar line lengths
rts_lengths = load("../data/RTS-96\ Data/line_lengths.jld", "line_lengths")
nz_rts_lengths = rts_lengths[find(rts_lengths)]

# sample nz_rts_lengths to generate polish line lengths:
line_lengths = zeros(length(lines))
for i in 1:length(lines)
    idx = rand(1:length(nz_rts_lengths))
    line_lengths[i] = nz_rts_lengths[idx]
end
show([mean(line_lengths),std(line_lengths)])

# do NOT uncomment unless you wish to permanently alter polish grid line lengths!
#save("../data/polish_line_lengths.jld","line_lengths",line_lengths)

[51559.67147847275,27550.81173951869]

## Testing

In [1]:
# Analysis:
include("../src/TemporalInstanton.jl")
using TemporalInstanton

In [1]:
# Analysis:
addprocs(4)
@everywhere include("../src/TemporalInstanton.jl")
@everywhere using TemporalInstanton

In [2]:
Ridx, Y,
Gp, Dp, Rp,
Sb, ref, lines,
res, reac, k,
line_lengths, line_conductors = load_polish_data();

In [3]:
time_intervals = 3 # thirty minutes
time_values = 0:60:600 # ten minutes in 1-min steps
int_length = 600. # seconds
G0 = [0.7*Gp;0.7*Gp;0.7*Gp]
D0 = [0.9*Dp;0.9*Dp;0.9*Dp]
P0 = [Rp;1.1*Rp;1.2*Rp]

# time_intervals = 6 # thirty minutes total
# time_values = 0:30:300 # five minutes in 30-sec steps
# int_length = 300. # seconds = 5 min
# G0 = [0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp]
# D0 = [0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp]
# P0 = [Rp;1.1*Rp;1.2*Rp;1.3*Rp;1.4*Rp;1.5*Rp]

# Thermal model parameters:
Tamb = 35. # C
T0 = 60. #46. # initial line steady-state temp

60.0

In [ ]:
@time score,x,θ,α,diffs,xopt = solve_temporal_instanton(
    Ridx,Y,G0,P0,D0,Sb,ref,lines,
    res,reac,k,line_lengths,line_conductors,Tamb,
    T0,int_length);